# Day 15: Dueling Generators

Here, you encounter a pair of dueling generators. The generators, called *generator A* and *generator B*, are trying to agree on a sequence of numbers. However, one of them is malfunctioning, and so the sequences don't always match.

As they do this, a *judge* waits for each of them to generate its next value, compares the lowest 16 bits of both values, and keeps track of the number of times those parts of the values match.

The generators both work on the same principle. To create its next value, a generator will take the previous value it produced, multiply it by a *factor* (generator A uses 16807; generator B uses 48271), and then keep the remainder of dividing that resulting product by 2147483647. That final remainder is the value it produces next.

To calculate each generator's first value, it instead uses a specific starting value as its "previous value" (as listed in your puzzle input).

For example, suppose that for starting values, generator A uses `65`, while generator B uses `8921`. Then, the first five pairs of generated values are:

    --Gen. A--  --Gen. B--
       1092455   430625591
    1181022009  1233683848
     245556042  1431495498
    1744312007   137874439
    1352636452   285222916

In binary, these pairs are (with generator A's value first in each pair):

    00000000000100001010101101100111
    00011001101010101101001100110111
    
    01000110011001001111011100111001
    01001001100010001000010110001000
    
    00001110101000101110001101001010
    01010101010100101110001101001010
    
    01100111111110000001011011000111
    00001000001101111100110000000111
    
    01010000100111111001100000100100
    00010001000000000010100000000100

Here, you can see that the lowest (here, rightmost) 16 bits of the third value match: `1110001101001010`. Because of this one match, after processing these five pairs, the judge would have added only 1 to its total.

To get a significant sample, the judge would like to consider *40 million* pairs. (In the example above, the judge would eventually find a total of `588` pairs that match in their lowest 16 bits.)

After 40 million pairs, what is the judge's final count?

## Solution

First, define the generator factors:

In [1]:
factor_a = 16807
factor_b = 48271

Then, a generator function for the generators ...

In [2]:
def generator(start, factor):
    current = start

    while True:
        current = (current * factor) % 2147483647
        yield current

The judge takes two generators, and a sample count, and returns the number of times the lowest 16 bits of generator a matched the lowest 16 bits of generator b.

In [3]:
def judge(a, b, samples=40000000):
    mask = (1 << 16) - 1  # This mask can be used to get the low 16 bits
    return sum((next(a) & mask) == (next(b) & mask) for _ in range(samples))

Then we can run the test cases from the challenge text:

In [4]:
test_a = 65
test_b = 8921

assert judge(generator(test_a, factor_a),
             generator(test_b, factor_b), 5) == 1
assert judge(generator(test_a, factor_a),
             generator(test_b, factor_b)) == 588

And finally we solve part 1 of the challenge:

In [5]:
start_a, start_b = list(
    int(line.strip().split()[-1]) for line in
    open('day15.txt').readlines()
)
judge(generator(start_a, factor_a),
      generator(start_b, factor_b))

626